# Virtual Doctor Chatbot with LangChain - Iteration 1

This Jupyter notebook demonstrates the creation of a virtual doctor chatbot using the LangChain framework and GPT-3.5. The chatbot is designed to interact with patients, providing responses based on the context of the conversation.

Components are:
1. **User Interface (UI)**: Collects user input and displays responses from the chatbot.
2. **Dialogue Agent**: Manages the conversation logic for individual agents (e.g., doctor and patient).
3. **Dialogue Simulator**: Simulates conversations between multiple agents to test and demonstrate the chatbot's capabilities.

Each component plays a crucial role in the overall interaction flow, from user input to intelligent response generation.

# Import and Setup

In [1]:
# Import necessary libraries
import os
import json
import random
import importlib.util
import sys
import unittest
from dotenv import load_dotenv
from typing import List, Tuple, Dict, Any
from langchain import LLMChain
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import AIMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.schema.output_parser import StrOutputParser
from reco_analysis.chatbot.chatbot import DialogueAgent, get_session_history, session_store, model
from reco_analysis.chatbot.prompts import system_message_doctor
from unittest.mock import MagicMock

2024-06-23 13:27:28.815 | INFO     | reco_analysis.config:<module>:11 - PROJ_ROOT path is: /Users/garykong/Library/CloudStorage/GoogleDrive-garykong91@gmail.com/My Drive/Schoolwork/recoverycompanion/reco_analysis


In [2]:
# Load API keys from .env file
# Replace this with the path to where your .env file is, which should contain openAI API keys as well as other necessary environment variables
# Env file should contain something like this:
# OPENAI_API_KEY=your openai api key
# OPENAI_ORG_ID=your org id
# LANGCHAIN_TRACING_V2='true'
# LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# LANGCHAIN_API_KEY=your langchain api key
# LANGCHAIN_PROJECT=the project name you want to use
load_dotenv("../.env") 

# Set environment variables for LangChain
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# System Messages

In [7]:
# The doctor system message has been loaded from the chatbot.prompts module
print(f"System message: {system_message_doctor}")

System message: 
You are a virtual doctor interacting with heart failure patients who have recently been discharged from the hospital. Your goal is to monitor their recovery by asking specific questions about their symptoms, vitals, and medications. Ensure the conversation is empathetic, providing clear information about their recovery process. 

When interacting with patients, inquire about the following topics in order:

Topic 1: Introduction and Open-Ended Symptom Inquiry
   - Greeting and Context: "Hello, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge."
   - Open-Ended Question: "Can you tell me how you've been feeling today? Have you noticed any new or worsening symptoms?"

Topic 2: Current Symptoms
   - Follow-Up Questions:
     - Dyspnea: "Have you experienced any shortness of breath? If yes, does it occur at rest, when walking, or when climbing stairs?"
     - Paroxysmal Nocturnal Dyspnea (PND): "Have you had sudden short

In [3]:
## Load patient data

# Specify the path to your JSON file
json_file_path = '../data/patients/patients_1.0.json'

# Open and read the JSON file
with open(json_file_path, 'r') as json_file:
    patients = json.load(json_file)

In [4]:
random_key = random.choice(list(patients.keys()))

In [5]:
print(patients[random_key]['prompt'])


    You are Jennifer Navarro, a patient who has been discharged after a hospital stay for heart failure. You are reporting your symptoms for a routine check-in with your doctor. Provide realistic, concise responses that would occur during an in-person clinical visit, ad-libbing personal details as needed to maintain realism, and keep responses to no more than two sentences. Include some filler words like 'um...' and 'ah...' to simulate natural conversation. Do not relay all information at once. 

    Use the profile below during the conversation:
    <input>
    Gender: Female
    Age: 86
    Race: Hispanic/Latino
    Marital status: Widowed
    Current symptoms: Chest pain, Dyspnea (report your symptoms in plain language, avoid medical terminology)
    Current emotional state: Normal (can improve based on interaction with the doctor)
    Current medications to report: Torsemide, Metoprolol (only mention a few at a time)
    Vital signs information:
    - Temperature: 97.8
    - Heart

In [6]:
system_message_patient = patients[random_key]['prompt']

# User Interface

The `UserInterface` class is responsible for managing user interactions with the chatbot. It collects user input and displays the chatbot's responses.

The class includes methods for:
- Collecting user input through the console.
- Displaying responses generated by the chatbot.

This simple interface allows users to interact with the chatbot in real-time.

In [7]:
class UserInterface:
    def __init__(self, agent_role='Doctor'):
        """
        Initialize the User Interface

        Args:
            agent_role (str, optional): The role of the agent. Defaults to 'Doctor'.
        """
        agent_role = agent_role.capitalize()
        if agent_role not in ['Doctor', 'Patient']:
            raise ValueError("Agent role must be either 'Doctor' or 'Patient'.")
        self.agent_role = agent_role  # By default, the agent is a doctor
            
    def collect_user_input(self):
        """
        Collects user input.

        Returns:
            str: The user input.
        """
        user_input = input("Enter user message. Enter 'exit' to stop chat: ")
        return user_input

    def display_response(self, response: str):
        """
        Displays the response to the user.

        Args:
            response (str): The response to display.
        """
        print(f"{self.agent_role}: {response}")

# Dialogue Agent

The `DialogueAgent` class is the core of the conversational logic. It represents either the doctor or the patient in our virtual doctor scenario.

Key Features:
- **Initialization**: Sets up the agent with a specified role (Doctor or Patient) and a system message to guide the conversation.
- **Memory Management**: Maintains the history of the conversation.
- **Message Handling**: Receives messages from other agents or users and generates appropriate responses using GPT-3.5.
- **Response Generation**: Uses LangChain's `LLMChain` to predict the next response based on the conversation history.

This class manages how each agent (doctor or patient) interacts during the conversation.

In [12]:
# The DialogueAgent class was imported from the chatbot.chatbot module. Here we just print the available attributes to check it is working
for attribute in dir(DialogueAgent()):
    if not attribute.startswith("__"):
        print(attribute)

ai_instruct
chain
generate_response
get_history
human_role
memory
model
prompt
receive
reset
role
send
session_id
system_message


........
----------------------------------------------------------------------
Ran 8 tests in 0.070s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>

### Automated Unit Tests

In [ ]:
# Unit tests for the chatbot are defined in the test_chatbot.py module. We will load the module and run the tests here.
# Define the path to the test module
test_module_path = '../reco_analysis/chatbot/test_chatbot.py'

# Load the module from the specified path
spec = importlib.util.spec_from_file_location("test_chatbot", test_module_path)
test_module = importlib.util.module_from_spec(spec)
sys.modules["test_chatbot"] = test_module
spec.loader.exec_module(test_module)

# Run the tests
unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromModule(test_module))

### Test Using List of User Messages

In [16]:
# Test function for the DialogueAgent (with a list of user messages)
def test_dialogue_agent(system_message, agent_role, user_messages):
    """
    Tests the DialogueAgent acting as a doctor/patient with a series of patient/doctor messages.
    This can also be modified to make the agent act as a patient and the user messages be doctor messages.

    Args:
        system_message (str): The initial system message for the agent
        agent_role (str): The role of the agent (either 'doctor' or 'patient')
        user_messages (List[str]): The list of user messages to simulate the conversation
        verbose (bool, optional): Whether to print verbose output. Defaults to False.
    """
    # Create the DialogueAgent instance for the doctor
    agent = DialogueAgent(
        role=agent_role,
        system_message=system_message,
        model=model,
    )

    # Define user and agent roles
    agent_role = agent_role.capitalize()
    user_role = "Patient" if agent_role == "Doctor" else "Doctor"
        
    # Simulate the conversation
    for msg in user_messages:
        # Doctor receives the patient's message
        agent.receive(message=msg)
        
        # Doctor sends a response
        response = agent.generate_response()
        
        # Print the response
        print(f"{user_role}: {msg}")
        print(f"{agent_role}: {response}\n")

    # Get the full conversation history
    history = agent.get_history()

    return history

In [17]:
# # Run the test function with the agent acting as a doctor
# test_dialogue_agent(
#    system_message = system_message_doctor,
#    agent_role = 'Doctor',
#    user_messages = [
#        "Hello, doctor. I've been feeling a bit dizzy lately.",
#        "I've also been experiencing some chest pain and shortness of breath.",
#        "I'm worried it might be related to my heart condition.",
#    ]
# )

Patient: Hello, doctor. I've been feeling a bit dizzy lately.
Doctor: Hello, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge. Can you tell me more about how you've been feeling? Have you noticed any new or worsening symptoms besides feeling dizzy?

Patient: I've also been experiencing some chest pain and shortness of breath.
Doctor: I'm sorry to hear that you're experiencing chest pain and shortness of breath. Could you tell me more about the chest pain? How would you describe it?

Patient: I'm worried it might be related to my heart condition.
Doctor: It's understandable to be concerned, especially with your heart condition. In terms of your symptoms, have you experienced any shortness of breath at rest, when walking, or when climbing stairs?



["Patient: Hello, doctor. I've been feeling a bit dizzy lately.",
 "Doctor: Hello, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge. Can you tell me more about how you've been feeling? Have you noticed any new or worsening symptoms besides feeling dizzy?",
 "Patient: I've also been experiencing some chest pain and shortness of breath.",
 "Doctor: I'm sorry to hear that you're experiencing chest pain and shortness of breath. Could you tell me more about the chest pain? How would you describe it?",
 "Patient: I'm worried it might be related to my heart condition.",
 "Doctor: It's understandable to be concerned, especially with your heart condition. In terms of your symptoms, have you experienced any shortness of breath at rest, when walking, or when climbing stairs?"]

In [19]:
# # Run the test function with the agent acting as a patient
#test_dialogue_agent(
#    system_message = system_message_patient,
#    agent_role = 'patient',
#    user_messages = [
#        "Hi there. How are you doing today?",
#        "What symptoms are you currently experiencing?",
#        "Have you been taking your medications regularly?",
#    ]
#)

### Test Using User Input Through UI Class

In [18]:
# Test function for the DialogueAgent using the UserInterface (enabling user input)
def test_dialogue_with_ui(system_message, agent_role):
    """
    This function simulates a conversation between a virtual doctor and a patient using a user interface, whereby the user can input messages acting as the patient or the doctor.
    This can also be modified to simulate a conversation whereby the agent acts as the patient and the user acts as the doctor.

    Args:
        system_message (str): The initial system message for the agent
        agent_role (str): The role of the agent (either 'doctor' or 'patient')
    """
    # Create the DialogueAgent instance for the doctor
    agent = DialogueAgent(
        role=agent_role,
        system_message=system_message,
        model=model,
    )

    # Define user and agent roles
    agent_role = agent_role.capitalize()
    human_role = "Patient" if agent_role == "Doctor" else "Doctor"
    
    # Create the UserInterface instance
    ui = UserInterface(agent_role=agent_role)
    
    # Simulate the interaction
    while True:
        # Collect user input
        user_input = ui.collect_user_input()
        print(f"{human_role}: {user_input}")
        
        if user_input.lower() == 'exit':
            break
        
        # Doctor agent receives the user's message
        agent.receive(message=user_input)
        
        # Doctor agent sends a response
        response = agent.generate_response()
        
        # Display the doctor's response
        ui.display_response(response)
    
    # Get the full conversation history
    history = agent.get_history()
    
    return history

In [21]:
# Run the test function with the agent acting as a doctor
#test_dialogue_with_ui(
#    system_message=system_message_doctor,
#    agent_role='Doctor',
#)

In [19]:
# Run the test function with the agent acting as a patient
test_dialogue_with_ui(
   system_message=system_message_patient,
   agent_role='Patient'
)

Doctor: Hi doctor just testing this works
Patient: That's fine. Um... my heart rate is usually around 92 beats per minute.
Doctor: exit


['Doctor: Hi doctor just testing this works',
 "Patient: That's fine. Um... my heart rate is usually around 92 beats per minute."]

# Dialogue Simulator

The `DialogueSimulator` class simulates a conversation flow between two agents (a doctor and a patient). It does so by managing the turn-taking process and message exchanges between agents.

Key features:
- **Initialization**: Takes a list of `DialogueAgent` instances and prepares the simulator for running conversations.
- **Turn Selection**: Determines which agent speaks next based on a simple turn-taking mechanism.

This class is essential for running automated tests and simulations of conversations between different agents.

In [14]:
class DialogueSimulator():
    def __init__(self, agents: List[DialogueAgent]):
        """
        Initialize the DialogueSimulator with a list of agents and the starting agent.

        Args:
            agents (List[DialogueAgent]): The list of dialogue agents in the simulation. Must contain exactly 2 agents (a doctor and a patient).
        """        
        
        # Check if the number of agents is exactly 2
        self.agents = agents
        if len(self.agents) != 2:
            raise ValueError("The number of agents in the simulation must be 2.")
        
        # Define the doctor and patient agents
        self.doctor_idx, self.patient_idx = self.define_doctor_and_patient_indices()
        self.doctor_agent = self.agents[self.doctor_idx]
        self.patient_agent = self.agents[self.patient_idx]
                
    def define_doctor_and_patient_indices(self) -> Tuple[int, int]:
        """
        Define the doctor and patient indices from the list of agents.
        """
        for i, agent in enumerate(self.agents):
            if agent.role == "Doctor":
                doctor_idx = i
            elif agent.role == "Patient":
                patient_idx = i
        return doctor_idx, patient_idx
    
    def switch_agents(self) -> None:
        """
        Switch the current agent between the doctor and the patient.
        """
        self.current_agent = self.doctor_agent if self.current_agent == self.patient_agent else self.patient_agent

    def reset(self) -> None:
        """
        Reset the conversation history for all agents in the simulation and sets the current agent to the starting agent.
        """
        for agent in self.agents:
            agent.reset()
        self.current_agent = self.doctor_agent if self.starting_agent == "Doctor" else self.patient_agent

    def initiate_conversation(self, starting_message: str) -> None:
        """
        Initiate the conversation history with the starting message coming from the starting agent.
        """
        self.reset()

        speaker = self.doctor_agent if self.starting_agent == "Doctor" else self.patient_agent
        receiver = self.patient_agent if self.starting_agent == "Doctor" else self.doctor_agent

        speaker.send(starting_message)
        receiver.receive(starting_message)

        self.switch_agents()

    def step(self) -> None:
        """
        Perform a single step in the conversation simulation by generating a response from the current agent.
        """
        # Define the speaker and receiver
        speaker = self.current_agent
        receiver = self.doctor_agent if speaker == self.patient_agent else self.patient_agent

        # Generate a response from the speaker
        message = speaker.generate_response()
        print(f"{speaker.role}: {message}")

        # Receive the response from the speaker
        receiver.receive(message)

        # Switch the current agent
        self.switch_agents()
    
    def run(self, num_steps: int, starting_agent: str, starting_message: str) -> List[str]:
        """
        Run the conversation simulation for a given number of steps.

        Args:
            num_steps (int): The number of steps to run the simulation.
            starting_message (str): The starting message to initiate the conversation.
        """
        # Set the current agent to the starting agent
        self.starting_agent = starting_agent.capitalize()
        if self.starting_agent not in ["Doctor", "Patient"]:
            raise ValueError("Starting agent must be either 'Doctor' or 'Patient'")       
        self.current_agent = self.doctor_agent if self.starting_agent == "Doctor" else self.patient_agent
          
        # Inject the initial message into the memory of the conversation
        self.initiate_conversation(starting_message)

        # Run the simulation for the specified number of steps
        for _ in range(num_steps):
            self.step()

        # Get history from the starting agent
        history = self.doctor_agent.get_history() if self.starting_agent == "Doctor" else self.patient_agent.get_history()
        
        return history

In [20]:
def run_dialogue_simulator(
        system_message_doctor=system_message_doctor,
        system_message_patient=system_message_patient,
        num_steps=10,
        starting_agent="Doctor",
        starting_message="Hi there! How are you feeling today?"
):
    """
    Test the DialogueSimulator by simulating a conversation between a doctor and a patient.

    Args:
        system_message_doctor (str): The system message for the doctor agent.
        system_message_patient (str): The system message for the patient agent.
        num_steps (int): The number of steps to run the simulation.
        starting_agent (str): The starting agent for the conversation.
        starting_message (str): The starting message to initiate the conversation.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.
    """

    # Create DialogueAgent instances for the doctor and patient
    doctor_agent = DialogueAgent(role='Doctor', system_message=system_message_doctor, model=model)
    patient_agent = DialogueAgent(role='Patient', system_message=system_message_patient, model=model)

    simulator = DialogueSimulator(agents=[doctor_agent, patient_agent])

    # Run the dialogue simulation
    conversation_history = simulator.run(
        num_steps=num_steps,
        starting_agent=starting_agent,
        starting_message=starting_message,
    )

    # Return the conversation history
    return conversation_history

In [21]:
## Test with doctor as the starting agent
# convo = run_dialogue_simulator(
#    num_steps=50,
#    starting_agent="Doctor",
#    starting_message="Hello, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge."
#)

In [22]:
convo = run_dialogue_simulator(
    num_steps=50,
    starting_agent="Doctor",
    starting_message=f"Hello {patients[random_key]['name']}, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge.",
    system_message_patient = patients[random_key]['prompt']
)
patients[random_key]['chat_transcript'] = convo

Patient: Um... I've been feeling okay, doctor. Just a bit of chest pain, maybe a 2 out of 10.
Doctor: I'm sorry to hear about the chest pain, Jennifer. Let's address that. Have you experienced any shortness of breath, especially when resting, walking, or climbing stairs?
Patient: Um... yes, doctor. I've been feeling a bit short of breath when I walk around the house. It's been a little harder than usual.
Doctor: Thank you for sharing that, Jennifer. Have you also had sudden shortness of breath that wakes you up at night?
Patient: No, doctor, I haven't had any sudden shortness of breath waking me up at night. It's mainly when I'm up and about during the day.
Doctor: Based on your responses, it seems you are experiencing some shortness of breath while walking around the house but not waking up at night suddenly. Do you need to prop yourself up with pillows to breathe comfortably while lying down?
Patient: No, doctor, I don't need to prop myself up with pillows at night. I usually sleep l

In [23]:
print(patients[random_key]['chat_transcript'])

["Doctor: Hello Jennifer Navarro, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge.", "Patient: Um... I've been feeling okay, doctor. Just a bit of chest pain, maybe a 2 out of 10.", "Doctor: I'm sorry to hear about the chest pain, Jennifer. Let's address that. Have you experienced any shortness of breath, especially when resting, walking, or climbing stairs?", "Patient: Um... yes, doctor. I've been feeling a bit short of breath when I walk around the house. It's been a little harder than usual.", 'Doctor: Thank you for sharing that, Jennifer. Have you also had sudden shortness of breath that wakes you up at night?', "Patient: No, doctor, I haven't had any sudden shortness of breath waking me up at night. It's mainly when I'm up and about during the day.", 'Doctor: Based on your responses, it seems you are experiencing some shortness of breath while walking around the house but not waking up at night suddenly. Do you need to prop y

In [24]:
for key in patients.keys():
    convo = run_dialogue_simulator(
        num_steps=50,
        starting_agent="Doctor",
        starting_message=f"Hello {patients[key]['name']}, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge.",
        system_message_patient = patients[key]['prompt']
    )
    patients[key]['chat_transcript'] = convo

Patient: Ah, um... well, I've been feeling a bit tired lately, and my legs have been swelling up a bit.
Doctor: Based on your responses, it seems you've been feeling tired and experiencing swelling in your legs. Let's address these symptoms further. Have you experienced any shortness of breath recently?
Patient: Ah, yes, um... I do get short of breath when I walk around the house or go up the stairs. It's been a bit harder than usual.
Doctor: I'm sorry to hear that you've been experiencing shortness of breath with activity. Have you had sudden shortness of breath that wakes you up at night?
Patient: No, um... I haven't had any sudden shortness of breath at night. It's mainly when I'm up and moving around.
Doctor: It's good to know that you haven't experienced sudden shortness of breath at night. Do you find yourself needing to prop yourself up with pillows to breathe comfortably while lying down?
Patient: No, I don't need extra pillows to sleep comfortably. I usually manage alright onc

In [ ]:
### WRITE TO JSON FILE

file_path = '../data/patients/patients_1.0_with_transcripts.json'

with open(file_path, 'w') as json_file:
    json.dump(patients, json_file)